Here I want to make a simple recommender system to find the similarity between shows, users and to help me predict whether a user will enjoy a particular anime.

In [51]:
# Import relevant libraries 

import pandas as pd
import numpy as np
import scipy as sp
import sklearn
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import operator
%matplotlib inline

In [128]:
anime = pd.read_csv('animes.csv').drop(columns=['synopsis','link'])
# anime = anime.drop(x if 'Hentai' x in anime.genre)
rating = pd.read_csv('reviews.csv').drop(columns=['anime_uid','text','link','scores'])
anime.head()

,uid,title,genre,aired,episodes,members,popularity,ranked,score,img_url
0,28891,Haikyuu!! Second Season,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...
1,23273,Shigatsu wa Kimi no Uso,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",22.0,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...
2,34599,Made in Abyss,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","Jul 7, 2017 to Sep 29, 2017",13.0,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...
3,5114,Fullmetal Alchemist: Brotherhood,"['Action', 'Military', 'Adventure', 'Comedy', ...","Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...
4,31758,Kizumonogatari III: Reiketsu-hen,"['Action', 'Mystery', 'Supernatural', 'Vampire']","Jan 6, 2017",1.0,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...


In [ ]:
rating.head()

Join the two dataframes on the anime_id columns

In [77]:
# Merging anime_show and rating dataframe with inner join
merged = rating.merge(anime, on = 'uid', suffixes= ['_user', ''], how='inner')
# Renaming column
merged.rename(columns = {'uid':'anime_id'}, inplace = True)
merged.rename(columns = {'score_user':'user_rating'}, inplace = True)
# Dropping extra column
print(merged.shape)
merged.head()

(12783, 12)


,anime_id,profile,user_rating,title,genre,aired,episodes,members,popularity,ranked,score,img_url
0,29323,Slushpuppy282,7,"Oyaji no, Imo no Kamisama.",['Slice of Life'],"Dec 31, 2014",1.0,360,11732,8664.0,5.9,https://cdn.myanimelist.net/images/anime/2/705...
1,29323,Slushpuppy282,7,"Oyaji no, Imo no Kamisama.",['Slice of Life'],"Dec 31, 2014",1.0,360,11732,8664.0,5.9,https://cdn.myanimelist.net/images/anime/2/705...
2,30968,ParaParaJMo,9,Kokoro no Catchball,"['Kids', 'Sports']",2005,1.0,100,15323,12764.0,6.7,https://cdn.myanimelist.net/images/anime/2/745...
3,30968,ParaParaJMo,9,Kokoro no Catchball,"['Kids', 'Sports']",2005,1.0,100,15323,12764.0,6.7,https://cdn.myanimelist.net/images/anime/2/745...
4,38440,Jolon,9,Shikizakura,"['Action', 'Sci-Fi', 'Drama']",2021 to ?,12.0,1419,8859,NaN,NaN,https://cdn.myanimelist.net/images/anime/1203/...


In [78]:
merged2 = merged.dropna()
print(merged2.shape)
merged2.head()

(10636, 12)


,anime_id,profile,user_rating,title,genre,aired,episodes,members,popularity,ranked,score,img_url
0,29323,Slushpuppy282,7,"Oyaji no, Imo no Kamisama.",['Slice of Life'],"Dec 31, 2014",1.0,360,11732,8664.0,5.9,https://cdn.myanimelist.net/images/anime/2/705...
1,29323,Slushpuppy282,7,"Oyaji no, Imo no Kamisama.",['Slice of Life'],"Dec 31, 2014",1.0,360,11732,8664.0,5.9,https://cdn.myanimelist.net/images/anime/2/705...
2,30968,ParaParaJMo,9,Kokoro no Catchball,"['Kids', 'Sports']",2005,1.0,100,15323,12764.0,6.7,https://cdn.myanimelist.net/images/anime/2/745...
3,30968,ParaParaJMo,9,Kokoro no Catchball,"['Kids', 'Sports']",2005,1.0,100,15323,12764.0,6.7,https://cdn.myanimelist.net/images/anime/2/745...
8,27501,Mimi_Taylor,9,Sore Ike! Anpanman: Anpanman to Tanoshii Nakam...,"['Kids', 'Adventure', 'Fantasy']","Jul 24, 1999",1.0,171,13933,13953.0,7.1,https://cdn.myanimelist.net/images/anime/9/666...


Pivoting table of users on one axis and tv show names along the other. This helps us in defining the similarity between users and shows to better predict who will like what.

In [100]:
piv = merged.pivot_table(index=['profile'], columns=['title'], values='user_rating',fill_value=0)

In [101]:
print(piv.shape)
piv.head()

(3947, 7468)


title,"""Aesop"" no Ohanashi yori: Ushi to Kaeru, Yokubatta Inu","""Bungaku Shoujo"" Memoire","""Eiji""","""Eiyuu"" Kaitai","""Kinako"" x Mameshiba Movie","""Parade"" de Satie","""Tokyo""",#Compass,(a long day of) Mr. Calpaccio,.hack//G.U. Trilogy: Parody Mode,...,lilac (bombs Jun Togawa),s.CRY.ed,the FLY BanD!,xxxHOLiC,xxxHOLiC Kei,xxxHOLiC Movie: Manatsu no Yoru no Yume,xxxHOLiC Shunmuki,Üks Uks,ēlDLIVE,◯
profile,,,,,,,,,,,,,,,,,,,,,
--skeletor--,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
-Alice-,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
-Animewatcher-,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
-Exo-,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
-FAWKYOURFACE-,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Transposing the Matrix (eg. pivoted table)

In [102]:
X = piv.T
X.shape

(7468, 3947)

Decomposing the Matrix - Lowering the total features

In [103]:
# Truncates matrix in 12 Synthetic features
SVD = TruncatedSVD(n_components=12,random_state=12)

resultant_matrix = SVD.fit_transform(X)

resultant_matrix.shape

(7468, 12)

Generating a Correlation Matrix

In [104]:
corr_mat = np.corrcoef(resultant_matrix)
corr_mat.shape


(7468, 7468)

In [105]:
anime_titles = piv.columns
anime_list = list(anime_titles)


In [124]:
naruto = anime_list.index('naruto'.title())
print(naruto_corr)
corr_naruto = corr_mat[int(naruto)]
corr_naruto.shape

4471


(7468,)

Recommending top 10% of Hightly Correlated Anime

In [125]:
list(anime_titles[(corr_naruto<1.0)&(corr_naruto>0.9)])

['Anzen no Shiteiseki Child Seat',
 'Blood: The Last Vampire',
 'Bokura no Ashita: Freeter no Genjou ni Kansuru Wakamono e no Shuuchi Kouhou Jigyou',
 'Chameko no Ichinichi',
 'Discode: Ijou Seiai',
 'Fairy Tail Movie 1: Houou no Miko',
 'Fushigi Yuugi: Eikouden',
 'Garo: Guren no Tsuki',
 'Given Movie',
 'High School Fleet OVA',
 'Inazuma Eleven Go: Chrono Stone',
 'Joker Game',
 'Kagee Aesop Monogatari',
 'Kimi no Na wa.',
 'Lupin III: Lupin wa Ima mo Moete Iru ka?',
 'Macross 7 Plus',
 'Manaria Friends',
 'Mardock Scramble: The Third Exhaust',
 'Meitantei Rascal',
 'Mobile Suit Gundam UC: A Phantom World',
 'Nana',
 'Nekoronde Mite ne.',
 'New My Tear CL Series',
 'Nurse Witch Komugi-chan Magikarte',
 'Oh Yoko!',
 'Osomatsu-kun (1988)',
 'Overlord Movie: Manner Movie',
 'Panda no Blues',
 'Scrapped Princess',
 'Sennen no Yakusoku',
 'Speed Grapher',
 'Starry☆Sky',
 'Tamayura: Hitotose',
 'Tamayura: More Aggressive',
 'Tobe! Kujira no Peek',
 'Tsundere Inran Shoujo Sukumi',
 'Uchuu S

Function to recommend the top correlated Anime, (top inputted percentage)

In [90]:
def rec_anime(x,float):
    x_corr = anime_list.index(f'{x.title()}')
    x_corr
    corr_x = corr_mat[x_corr]
    print(f'Top {int(float*100)} Percentile Correlational match to Anime: {x.title()}')
    corr_anime = list(anime_titles[(corr_x < 1.0) & (corr_x > (1.0-float))])
    return corr_anime


In [98]:
rec_anime('naruto',.1)

Top 10 Percentile Correlational match to Anime: Naruto


['Allison to Lillia',
 'Anata mo Robot ni Nareru feat. Kamome Jidou Gasshoudan',
 'Ashita no Nadja',
 'Boogiepop wa Warawanai',
 'Chouon Senshi Borgman',
 'Da Zuiba Dudu',
 'Deadman Wonderland',
 'Dogs: Bullets & Carnage',
 'Dragon League',
 'Fukuyama Gekijou: Natsu no Himitsu',
 'Ganbare Genki',
 'Ganbare! Moudouken Serve',
 'Gegege no Kitarou (2007)',
 'Getter Robo',
 'Guilty Crown Kiseki: Reassortment',
 'Hakuouki Hekketsuroku',
 'Huckleberry no Bouken (Movie)',
 'Jormungand: Perfect Order Recap',
 'Kenritsu Chikyuu Boueigun',
 'Kiddy GiRL-AND Pilot',
 'Kikou Sennyo Rouran',
 'Kindaichi Shounen no Jikenbo Movie 1: Operazakan - Aratanaru Satsujin',
 'Kusatta Kyoushi no Houteishiki',
 'Kyoto Animation: Megane-hen',
 'Kyoto Animation: Suiei-hen',
 'Ladies versus Butlers! Specials',
 'Ling Yu 2nd Season',
 'Ling Yu: Di San Ji',
 'Lupin III: Dead or Alive',
 'Machine Robo: Leina, The Legend of Wolf Blade',
 'Maetel Legend',
 'Magic Kaito',
 'Manga Aesop Monogatari (TV)',
 'Maryuu Senki',

In [99]:
rec_anime('one piece',0.1)

ValueError: 'One Piece' is not in list